# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [76]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [77]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

In [78]:
# print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [79]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [80]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

799


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [81]:
# Below is the connection to a Cassandra instance my local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [82]:
# Below code will create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [83]:
# Specified keyspace name as : udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### Solution: Here I have created a primary key on sessionid and iteminsession, sessionid is the partioning key which will partition the data in way so that all the records for same session id will be stored in same partition. Below are the three steps solution 1.1. Create Table,  1.2 Insert the Data into Table and 1.3 Select query to validate the data

### 1.1 Create the table

In [84]:
query = "CREATE TABLE IF NOT EXISTS artist_song_title_by_songs_length"
query = query + "(sessionId int,itemInSession int,artist text,firstName text,gender varchar,lastName text,length decimal,level text, \
location text,song text,userId int,PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### 1.2 Insert the data

In [85]:
# Below code will load the data from CSV files into artist_song_title_by_songs_length table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:                      
        query = "INSERT INTO artist_song_title_by_songs_length(sessionId,itemInSession,artist,firstName,gender,lastName,length,level,location,song,userId)"
        query = query + " VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)"        
         ## map each column in the INSERT statement.  
        session.execute(query,(int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9],int(line[10])))

### 1.3 Select query to validate the output

In [86]:
query = "select artist,song,length from artist_song_title_by_songs_length WHERE sessionId=486 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

In Flames My sweet shadow 278.69995


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Solution: Here I have created a primary key on userid and sessionid, userid is the partioning key which will partition the data in way so that all the records for same session id will be stored in same partition. itemInSession is the clustering key by which it will do the ordering of the data. Below are the three steps solution 2.1. Create Table,  2.2 Insert the Data into Table and 2.3 Select query to validate the data

### 2.1 Create the table

In [87]:
query = "CREATE TABLE IF NOT EXISTS artist_songs_by_users"
query = query + "(userId int,sessionId int,itemInSession int,artist text,firstName text,gender varchar,lastName text,length decimal,level text, \
location text,song text,PRIMARY KEY ((userId,sessionId),itemInSession) \
) WITH CLUSTERING ORDER BY (itemInSession ASC);"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### 2.2 Insert the data

In [88]:
# Below code will load the data from CSV files into artist_songs_by_users table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:     
        ## Prepare the insert statement
        query = "INSERT INTO artist_songs_by_users(userId,sessionId,itemInSession,artist,firstName,gender,lastName,length,level,location,song)"
        query = query + " VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)"
        ## map each column in the INSERT statement.        
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9]))

### 2.3 Select query to validate the output

In [89]:
query = "select artist,song,firstname,lastname from artist_songs_by_users WHERE userid=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist,row.song,row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Solution: Here I have created a primary key on song, userid and sessionid which is composite primary key, song and userid both are the partioning key which will partition the data in way so that all the records for same session id will be stored in same partition.Below are the three steps solution 3.1. Create Table,  3.2 Insert the Data into Table and 3.3 Select query to validate the data

### 3.1 Create the table

In [90]:
query = "CREATE TABLE IF NOT EXISTS songs_listen_by_users"
query = query + "(song text,userId int,sessionId int,artist text,firstName text,gender varchar,itemInSession int,lastName text,length decimal,level text, \
location text,PRIMARY KEY (song))"
try:
    session.execute(query)
except Exception as e:
    print(e)   

### 3.2 Insert the data

In [91]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:      
        query = "INSERT INTO songs_listen_by_users(song,userId,sessionId,artist,firstName,gender,itemInSession,lastName,length,level,location)"
        query = query + " VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)"
          ## map each column in the INSERT statement.  
        session.execute(query,(line[9],int(line[10]),int(line[8]),line[0],line[1],line[2],int(line[3]),line[4],float(line[5]),line[6],line[7]))

### 3.3 Select query to validate the output

In [92]:
query = "select firstName,lastName from songs_listen_by_users WHERE song='You''re The One'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)

Kate Harrell


### Drop the tables before closing out the sessions

In [93]:
## Drop the table before closing out the sessions

In [94]:
query = "drop table artist_song_title_by_songs_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [95]:
query = "drop table artist_songs_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [96]:
query = "drop table songs_listen_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [97]:
session.shutdown()
cluster.shutdown()